In [1]:
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

In [2]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

In [3]:
# recursivejson.py
# https://hackersandslackers.com/extract-data-from-complex-json-python/

def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results


In [ ]:
from bs4 import BeautifulSoup
import re
import csv
import time

#where to put the data in csv
outfile = open('niche.csv','w')
writer=csv.writer(outfile)

#column names
writer.writerow(['ranking', 'name', 'African American', 'Asian', 'Hispanic', 'International', 'Multiracial', 'Native American','Pacific Islander', 'Unknown', 'White'])

#Pages 1-37 are ranked universities
#Iterate through api requests
for x in range(1, 37):
    r = requests.get('https://www.niche.com/api/renaissance/results/?type=private&type=public&listURL=best-colleges&page='+str(x)+'&searchType=college')
    r = r.text
    data = json.loads(r)
    #getting to deeply nested data of entities, where university info is
    data = data["entities"]
    
    #get name and url for all universities in that page
    names = extract_values(data, 'name')
    urls = extract_values(data, 'url')
    for idx, url in enumerate(urls):
        if "https" in url:
            del urls[idx]
    
    rankings =  extract_values(data, 'ordinal')
    collegeRankings = dict(zip(names, zip(rankings, urls)))
    #collegeRankings
    
    #get information from unique link for each of the colleges from the page
    for name,value in collegeRankings.items():
        if value[0] < 881:
            ranking = value[0]
            link = value[1]
            temparray = []
            temparray.append(ranking)
            temparray.append(link)
            r = requests.get("https://www.niche.com/colleges/"+link+"/students/")
            soup = BeautifulSoup(r.text)
            for idx, percentage in enumerate(soup.findAll('div', attrs={'class': 'fact__table__row__value'})):
                if idx > 13:
                    temparray.append(percentage.text)
            writer.writerow(temparray)
    time.sleep(60)



In [4]:
from bs4 import BeautifulSoup
import re
import csv
import time

#where to put the data in csv
outfile = open('niche.csv','w')
writer=csv.writer(outfile)

#column names
writer.writerow(['ranking', 'name', 'undergradpop'])

#Pages 1-37 are ranked universities
#Iterate through api requests
for x in range(29, 37):
    r = requests.get('https://www.niche.com/api/renaissance/results/?type=private&type=public&listURL=best-colleges&page='+str(x)+'&searchType=college')
    r = r.text
    data = json.loads(r)
    #getting to deeply nested data of entities, where university info is
    data = data["entities"]
    
    #get name and url for all universities in that page
    names = extract_values(data, 'name')
    urls = extract_values(data, 'url')
    for idx, url in enumerate(urls):
        if "https" in url:
            del urls[idx]
    
    rankings =  extract_values(data, 'ordinal')
    collegeRankings = dict(zip(names, zip(rankings, urls)))
    #collegeRankings
    
    #get information from unique link for each of the colleges from the page
    for name,value in collegeRankings.items():
        if value[0] < 881:
            ranking = value[0]
            link = value[1]
            temparray = []
            temparray.append(ranking)
            temparray.append(link)
            r = requests.get("https://www.niche.com/colleges/"+link+"")
            soup = BeautifulSoup(r.content, 'html.parser')
            
            for idx, value in enumerate(soup.findAll('div', attrs={'class': 'scalar__value'})):
                if (idx == 13) & ("Undergrads" in value.text):
                    value = BeautifulSoup(value.text, 'html.parser')
                    value = str(value).rstrip("Undergrads")
                    value = int(value.replace(",", ""))
                    print(value)
                    #print(value.children[0].innerHTML)
                    temparray.append(value)
            writer.writerow(temparray)
            
            
    print("here")
    time.sleep(60)


7125
3854
6559
9268
24869
2478
1736
3828
727
4910
1129
760
here
2411
15036
6419
962
5671
7621
2215
here
1843
4210
1386
13056
4138
8429
here
7285
8469
2473
6968
7846
1380
2716
1785
here
23491
1449
7053
9021
7605
1472
4199
7779
4418
7863
here
6588
1462
7407
6109
8962
983
654
12549
1180
23097
here
3312
1497
624
1276
9455
4753
1397
here
2332
16381
619
here


In [ ]:
'''
import requests
from bs4 import BeautifulSoup
import re
import csv

categories=['African American', 'Asian', 'Hispanic', 'International', 'Multiracial', 'Native American','Pacific Islander', 'Unknown', 'White']
datadict = {}



for name,value in collegeRankings.items():
    link = value[1]
    temparray = []
    r = requests.get("https://www.niche.com/colleges/"+link+"/students/")
    soup = BeautifulSoup(r.text)
    for idx, percentage in enumerate(soup.findAll('div', attrs={'class': 'fact__table__row__value'})):
        if idx > 13:
            temparray.append(percentage.text)
    datadict[name] = temparray

df = pd.DataFrame.from_dict(datadict, orient='index', columns=categories)
df
'''